In [ ]:
import duckdb
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

############################################
# CONFIG
############################################

CSV_FILE = "cdac_timestamp_power_deviation_dataset.csv"
TABLE_NAME = "power_logs"
MODEL_NAME = "defog/sqlcoder-7b-2"  # use 7b outside Colab

START_TIME = "06-06-2025 11:40"
END_TIME   = "06-06-2025 13:40"


In [ ]:

############################################
# STEP 1: LOAD CSV WITH PANDAS (SAFE PARSE)
############################################

df = pd.read_csv(CSV_FILE)
df.columns = ["start_datetime", "power_deviation"]

df["start_datetime"] = pd.to_datetime(
    df["start_datetime"],
    format="%d-%m-%Y %H:%M",
    errors="coerce"
)

df["power_deviation"] = pd.to_numeric(
    df["power_deviation"],
    errors="coerce"
)

df = df.dropna()

print("\n[1] CSV loaded and parsed correctly")
print(df.head())



[1] CSV loaded and parsed correctly
          start_datetime  power_deviation
3763 2025-06-06 11:40:00             24.8
3764 2025-06-06 11:50:00             19.4
3765 2025-06-06 12:00:00             22.7
3766 2025-06-06 12:10:00             18.3
3767 2025-06-06 12:20:00             15.3


In [ ]:

############################################
# STEP 2: LOAD INTO DUCKDB
############################################

con = duckdb.connect(database=":memory:")

con.execute(f"""
    CREATE TABLE {TABLE_NAME} (
        start_datetime TIMESTAMP,
        power_deviation DOUBLE
    )
""")

con.register("df_view", df)

con.execute(f"""
    INSERT INTO {TABLE_NAME}
    SELECT start_datetime, power_deviation
    FROM df_view
""")

print("\n[2] Data loaded into DuckDB")



[2] Data loaded into DuckDB


In [ ]:

############################################
# STEP 3: LOAD SQLCODER
############################################

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto"
)

model.eval()
print("\n[3] SQLCoder loaded")

############################################
# STEP 4: USER QUESTION
############################################

user_question = f"""
What is the average power deviation between
'{START_TIME}' and '{END_TIME}'?
"""


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/515 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/691 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]


[3] SQLCoder loaded


In [ ]:

############################################
# STEP 5: PROMPT
############################################

schema = f"""
Table {TABLE_NAME} (
    start_datetime TIMESTAMP,
    power_deviation DOUBLE
)
"""

prompt = f"""
### Database Schema
{schema}

### Question
{user_question}

### SQL Query
"""

############################################
# STEP 6: GENERATE SQL
############################################

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

with torch.no_grad():
    output = model.generate(
        **inputs,
        max_new_tokens=150,
        temperature=0.0,
        do_sample=False,
        pad_token_id=tokenizer.eos_token_id
    )

generated = tokenizer.decode(output[0], skip_special_tokens=True)
sql_query = generated.split("### SQL Query")[-1].strip()

print("\n[4] GENERATED SQL:\n")
print(sql_query)


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



[4] GENERATED SQL:

SELECT AVG(pl.power_deviation) AS average_power_deviation FROM power_logs pl WHERE pl.start_datetime BETWEEN '2025-06-06 11:40' AND '2025-06-06 13:40';


In [ ]:

############################################
# STEP 7: EXECUTE SQL IN DUCKDB
############################################

sql_result = con.execute(sql_query).df()

print("\n[5] DUCKDB RESULT:\n")
print(sql_result)



[5] DUCKDB RESULT:

   average_power_deviation
0                11.892308


In [ ]:

############################################
# STEP 8: PANDAS VERIFICATION (GROUND TRUTH)
############################################

start_ts = pd.to_datetime(START_TIME, format="%d-%m-%Y %H:%M")
end_ts   = pd.to_datetime(END_TIME, format="%d-%m-%Y %H:%M")

filtered = df[
    (df["start_datetime"] >= start_ts) &
    (df["start_datetime"] <= end_ts)
]

pandas_avg = filtered["power_deviation"].mean()

print("\n[6] PANDAS VERIFICATION:")
print("Rows used:", len(filtered))
print("Average:", pandas_avg)

############################################
# STEP 9: ASSERT MATCH
############################################

duckdb_avg = sql_result.iloc[0, 0]

assert abs(duckdb_avg - pandas_avg) < 1e-9, "❌ MISMATCH BETWEEN SQL AND PANDAS"

print("\n✅ SQL and Pandas results MATCH PERFECTLY")



[6] PANDAS VERIFICATION:
Rows used: 13
Average: 11.892307692307694

✅ SQL and Pandas results MATCH PERFECTLY


In [1]:
pip install duckdb pandas torch transformers accelerate sentencepiece

In [2]:
from google.colab import files
files.upload()

Saving cdac_timestamp_power_deviation_dataset.csv to cdac_timestamp_power_deviation_dataset.csv


{'cdac_timestamp_power_deviation_dataset.csv': b'\xef\xbb\xbfStartDateUTC,PowerDeviation\r\n01-05-2025 22:00,\r\n01-05-2025 22:10,\r\n01-05-2025 22:20,\r\n01-05-2025 22:30,\r\n01-05-2025 22:40,\r\n01-05-2025 22:50,\r\n01-05-2025 23:00,\r\n01-05-2025 23:10,\r\n01-05-2025 23:20,\r\n01-05-2025 23:30,\r\n01-05-2025 23:40,\r\n01-05-2025 23:50,\r\n02-05-2025 00:00,\r\n02-05-2025 00:10,\r\n02-05-2025 00:20,\r\n02-05-2025 00:30,\r\n02-05-2025 00:40,\r\n02-05-2025 00:50,\r\n02-05-2025 01:00,\r\n02-05-2025 01:10,\r\n02-05-2025 01:20,\r\n02-05-2025 01:30,\r\n02-05-2025 01:40,\r\n02-05-2025 01:50,\r\n02-05-2025 02:00,\r\n02-05-2025 02:10,\r\n02-05-2025 02:20,\r\n02-05-2025 02:30,\r\n02-05-2025 02:40,\r\n02-05-2025 02:50,\r\n02-05-2025 03:00,\r\n02-05-2025 03:10,\r\n02-05-2025 03:20,\r\n02-05-2025 03:30,\r\n02-05-2025 03:40,\r\n02-05-2025 03:50,\r\n02-05-2025 04:00,\r\n02-05-2025 04:10,\r\n02-05-2025 04:20,\r\n02-05-2025 04:30,\r\n02-05-2025 04:40,\r\n02-05-2025 04:50,\r\n02-05-2025 05:00,\r\n02-05

In [3]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [4]:
import os

# Set Hugging Face cache to Google Drive
os.environ["HF_HOME"] = "/content/drive/MyDrive/huggingface"
os.environ["TRANSFORMERS_CACHE"] = "/content/drive/MyDrive/huggingface/models"


In [5]:
import duckdb
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

############################################################
# AUTO SCHEMA GENERATOR (CSV → DuckDB + LLM)
############################################################

def generate_duckdb_schema_from_csv(csv_file: str, table_name: str):
    df = pd.read_csv(csv_file)

    schema_fields = []
    datetime_formats = [
        "%d-%m-%Y %H:%M",
        "%Y-%m-%d %H:%M:%S",
        "%Y-%m-%d %H:%M"
    ]

    for col in df.columns:
        col_clean = col.strip().lower().replace(" ", "_")
        inferred_type = "VARCHAR"

        # Datetime detection
        for fmt in datetime_formats:
            try:
                parsed = pd.to_datetime(df[col], format=fmt, errors="raise")
                df[col] = parsed
                inferred_type = "TIMESTAMP"
                break
            except Exception:
                pass

        # Numeric detection
        if inferred_type == "VARCHAR":
            coerced = pd.to_numeric(df[col], errors="coerce")
            if coerced.notna().mean() > 0.9:
                df[col] = coerced
                inferred_type = "DOUBLE"

        schema_fields.append((col_clean, inferred_type))

    # Rename DataFrame columns to match SQL
    df.columns = [c.strip().lower().replace(" ", "_") for c in df.columns]

    # Schema for SQLCoder (description)
    llm_schema = "Table {} (\n{}\n)".format(
        table_name,
        ",\n".join([f"    {c} {t}" for c, t in schema_fields])
    )

    # Executable DuckDB SQL
    create_table_sql = "CREATE TABLE {} (\n{}\n);".format(
        table_name,
        ",\n".join([f"    {c} {t}" for c, t in schema_fields])
    )

    return llm_schema, create_table_sql, df


############################################################
# CONFIG
############################################################
CSV_FILE = "cdac_timestamp_power_deviation_dataset.csv"
TABLE_NAME = "power_logs"
MODEL_NAME = "defog/sqlcoder-7b-2"
CSV_FILE = "cdac_timestamp_power_deviation_dataset.csv"                 # user CSV
TABLE_NAME = "user_data"              # auto-created table
MODEL_NAME = "defog/sqlcoder-7b-2"      # use 7b outside Colab

############################################################
# STEP 1: AUTO SCHEMA + DATA
############################################################

llm_schema, create_table_sql, df = generate_duckdb_schema_from_csv(
    csv_file=CSV_FILE,
    table_name=TABLE_NAME
)

print("\n[1] AUTO-GENERATED SCHEMA (LLM):\n")
print(llm_schema)

print("\n[2] CREATE TABLE SQL (DuckDB):\n")
print(create_table_sql)

############################################################
# STEP 2: LOAD DATA INTO DUCKDB
############################################################

con = duckdb.connect(database=":memory:")
con.execute(create_table_sql)

con.register("df_view", df)
con.execute(f"""
    INSERT INTO {TABLE_NAME}
    SELECT * FROM df_view
""")

print("\n[3] Data loaded into DuckDB")

############################################################
# STEP 3: LOAD SQLCODER
############################################################

# tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
# model = AutoModelForCausalLM.from_pretrained(
#     MODEL_NAME,
#     torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
#     device_map="auto"
# )

# model.eval()
# print("✅ Model loaded")
# print("\n[4] SQLCoder loaded")


/usr/local/lib/python3.12/dist-packages/transformers/utils/hub.py:110: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(



[1] AUTO-GENERATED SCHEMA (LLM):

Table user_data (
    startdateutc TIMESTAMP,
    powerdeviation VARCHAR
)

[2] CREATE TABLE SQL (DuckDB):

CREATE TABLE user_data (
    startdateutc TIMESTAMP,
    powerdeviation VARCHAR
);

[3] Data loaded into DuckDB


In [29]:
from google.colab import drive
drive.mount("/content/drive")

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

MODEL_NAME = "defog/sqlcoder-7b-2"
SAVE_DIR = "/content/drive/MyDrive/models/sqlcoder-7b-2"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    device_map="auto",
    low_cpu_mem_usage=True
)

tokenizer.save_pretrained(SAVE_DIR)
model.save_pretrained(SAVE_DIR)

print("MODEL SAVED TO DRIVE")


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

✅ Model downloaded and loaded


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("defog/sqlcoder-7b-2")
model = AutoModelForCausalLM.from_pretrained(
    "defog/sqlcoder-7b-2",
    device_map="auto"
)

print("✅ Loaded without downloading")


Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

In [6]:
!ls /content/drive/MyDrive/huggingface

models	xet


In [7]:

############################################################
# STEP 4: USER QUESTION
############################################################

user_question = """
What is the average power deviation between
'06-06-2025 11:40' and '06-06-2025 13:40'?
"""

############################################################
# STEP 5: PROMPT SQLCODER
############################################################

prompt = f"""
### Database Schema
{llm_schema}

### Question
{user_question}

### SQL Query
"""


In [8]:

############################################################
# STEP 6: GENERATE SQL
############################################################

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

with torch.no_grad():
    output = model.generate(
        **inputs,
        max_new_tokens=150,
        temperature=0.0,
        do_sample=False,
        pad_token_id=tokenizer.eos_token_id
    )

generated = tokenizer.decode(output[0], skip_special_tokens=True)
sql_query = generated.split("### SQL Query")[-1].strip()

print("\n[5] GENERATED SQL:\n")
print(sql_query)

############################################################
# STEP 7: EXECUTE SQL
############################################################

result_df = con.execute(sql_query).df()

print("\n[6] QUERY RESULT:\n")
print(result_df)


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



[5] GENERATED SQL:

SELECT AVG(cast(u.powerdeviation AS FLOAT)) AS average_power_deviation FROM user_data u WHERE u.startdateutc BETWEEN '2025-06-06 11:40' AND '2025-06-06 13:40';

[6] QUERY RESULT:

   average_power_deviation
0                11.892308
